<b>Author: Ronay Ak</b>

In [9]:
import cudf
import pandas as pd
import cupy 
import math
import time
from datetime import datetime
import numpy as np
import pickle

In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [11]:
OUTPUT_BUCKET_FOLDER = "./preprocessed/"
DATA_BUCKET_FOLDER = "./dataset/"
#SPARK_TEMP_FOLDER = "/outbrain/spark-temp/"

Here some csv files are subsets of the originals due to large dataset size.

In [12]:
evaluation = False

Start loading csv files..

In [13]:
documents_meta_df = cudf.read_csv(DATA_BUCKET_FOLDER+ 'documents_meta.csv', dtype=['int32','int32','int32','str'], 
                                names=['document_id_doc', 'source_id', 'publisher_id', 'publish_time'], header=0)
print(documents_meta_df.shape)
documents_meta_df.dtypes

(2999334, 4)


document_id_doc     int32
source_id           int32
publisher_id        int32
publish_time       object
dtype: object

In [14]:
# APPLY DROPNA
print('Drop rows with empty "source_id"...')
documents_meta_df = documents_meta_df.dropna(subset=['source_id'])
documents_meta_df.shape

Drop rows with empty "source_id"...


(2996816, 4)

In [50]:
source_publishers_df = documents_meta_df[["source_id", "publisher_id"]].drop_duplicates()
source_publishers_df.shape

(14394, 2)

In [51]:
# the index numbers do not match with spark.
source_publishers_df.head()

,source_id,publisher_id
0,1,603
12867,2,85
12874,3,740
29583,4,213
29597,5,184


In [52]:
# get list of source_ids without publisher_id
source_ids_without_publisher_df = source_publishers_df.loc[source_publishers_df.loc[:, 'publisher_id'].isnull()]
new_publishers_df = source_ids_without_publisher_df.loc[:, ['source_id']]
new_publishers_df.shape

(5058, 1)

In [53]:
new_publishers_df.head()

,source_id
29846,7
29853,8
30995,16
33712,20
36964,26


In [54]:
# print('Get list of source_ids without publisher_id...')
# source_ids_without_publisher_df = source_publishers_df[source_publishers_df.publisher_id.isnull()]['source_id']
# new_publishers_df = source_ids_without_publisher_df.loc[:, ['source_id']]

In [55]:
source_ids_without_publisher_df.head()

,source_id,publisher_id
29846,7,null
29853,8,null
30995,16,null
33712,20,null
36964,26,null


In [56]:
print('Maximum value of publisher_id used so far...')
max_pub =source_publishers_df.dropna(subset='publisher_id')['publisher_id'].max()
max_pub

Maximum value of publisher_id used so far...


1263

NOTE: "source_id" column of new_publishers_df generated by spark is not in order. Here, with cudf it is in order.

In [57]:
# maximum value of publisher_id used so far
new_publisher_cnt = source_ids_without_publisher_df.shape[0]
# rows filled with new publisher_ids
new_publishers_df['publisher_id'] = np.arange(max_pub + 1, max_pub + 1 + new_publisher_cnt)
new_publishers_df.shape

(5058, 2)

In [58]:
new_publishers_df.head()

,source_id,publisher_id
29846,7,1264
29853,8,1265
30995,16,1266
33712,20,1267
36964,26,1268


In [59]:
new_publishers_df.source_id.max(), new_publishers_df.source_id.min()

(14366, 7)

In [60]:
new_publishers_df.dtypes

source_id       int32
publisher_id    int64
dtype: object

In [61]:
# old and new publishers merged
fixed_source_publishers_df = source_publishers_df.dropna(subset='publisher_id')
fixed_source_publishers_df = cudf.concat([fixed_source_publishers_df, new_publishers_df])

In [62]:
fixed_source_publishers_df.shape, documents_meta_df.shape

((14394, 2), (2996816, 4))

In [63]:
fixed_source_publishers_df.tail()

,source_id,publisher_id
2990886,14354,6317
2990902,14355,6318
2990906,14356,6319
2991236,14365,6320
2991239,14366,6321


In [64]:
print('Update documents_meta with new publishers...')

documents_meta_df = documents_meta_df.drop(columns='publisher_id').merge(fixed_source_publishers_df, on='source_id')
documents_meta_df.head()

Update documents_meta with new publishers...


,document_id_doc,source_id,publish_time,publisher_id
0,1250431,18,2016-04-04 16:00:00,1050
1,1173544,18,2016-03-22 00:00:00,1050
2,1206196,18,2016-03-29 10:00:00,1050
3,1263205,18,2016-04-06 14:00:00,1050
4,1248555,18,2016-04-04 12:00:00,1050


In [65]:
#publisher id is not same in spark dataframe.
documents_meta_df[documents_meta_df['source_id'] == 26]

,document_id_doc,source_id,publish_time,publisher_id
77284,2287601,26,2015-10-16 04:00:00,1268


In [73]:
documents_categories_df =  cudf.read_csv(
    DATA_BUCKET_FOLDER + 'documents_categories.csv',
    names=['document_id_cat', 'category_id', 'confidence_level_cat'],
    dtype=['int64', 'int64','float32'], header=0)

We need to convert cudf df to panda df to be able to do these ops. this takes long time.

In [48]:
#Grouping category_ids and confidence_levels..
documents_categories_df = documents_categories_df.to_pandas()
documents_categories_grouped_df = documents_categories_df.groupby('document_id_cat', 
    as_index=False)['category_id', 'confidence_level_cat'].agg(lambda x: list(x)).rename(
    columns={'category_id': 'category_id_list', 'confidence_level_cat': 'confidence_level_cat_list'})

In [29]:
documents_categories_grouped_df.head()

,document_id_cat,category_id_list,confidence_level_cat_list
0,1,"[1706, 1603]","[0.47122037410736084, 0.03585372492671013]"
1,2,"[1705, 1708]","[0.6000000238418579, 0.4000000059604645]"
2,3,"[2003, 1403]","[0.5228370428085327, 0.039781082421541214]"
3,4,"[2003, 2006]","[0.9199999570846558, 0.07000000029802322]"
4,5,"[2006, 2003]","[0.9199999570846558, 0.07000000029802322]"


In [88]:
#"Reading documents_topics...
documents_topics_df = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'documents_topics.csv',
    names=['document_id_top', 'topic_id', 'confidence_level_top'],
    dtype={'document_id_top': 'int64','topic_id': 'int64','confidence_level_top': 'float32'},
    skiprows=1)

In [ ]:
#"Grouping topic_ids and confidence_levels..."
documents_topics_df = documents_topics_df.to_pandas()
documents_topics_grouped_df = documents_topics_df.groupby('document_id_top', 
    as_index=False)['topic_id', 'confidence_level_top'].agg(lambda x: list(x)).rename(
    columns={'topic_id': 'topic_id_list', 'confidence_level_top': 'confidence_level_top_list'})

In [31]:
documents_topics_grouped_df.head()

,document_id_top,topic_id_list,confidence_level_top_list
0,1,"[252, 35, 150, 276, 137]","[0.020726216956973076, 0.012773293070495129, 0..."
1,2,[216],[0.013028857298195362]
2,3,"[276, 107, 176, 160, 260, 283, 202, 156, 104, ...","[0.029913833364844322, 0.027589114382863045, 0..."
3,4,"[102, 75, 16, 244, 97, 239]","[0.08417084068059921, 0.08329997211694717, 0.0..."
4,5,"[75, 102, 244, 143, 130, 202, 16, 44, 64]","[0.10655508935451508, 0.08646664023399353, 0.0..."


In [91]:
#Reading documents_entities..."
documents_entities_df = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'documents_entities.csv',
    names=['document_id_ent', 'entity_id', 'confidence_level_ent'],
    dtype={'document_id_ent': 'int64','entity_id': 'str','confidence_level_ent': 'float32'},
    skiprows=1)

In [ ]:
#Grouping entity_ids and confidence_levels...
documents_entities_df = documents_entities_df.to_pandas()
documents_entities_grouped_df = documents_entities_df.groupby('document_id_ent', 
    as_index=False)['entity_id', 'confidence_level_ent'].agg(lambda x: list(x)).rename(
    columns={'entity_id': 'entity_id_list', 'confidence_level_ent': 'confidence_level_ent_list'})

In [33]:
documents_entities_grouped_df.head()

,document_id_ent,entity_id_list,confidence_level_ent_list
0,2,"[e88e4cde6c6482ff01d4ad424904ffd1, a10e6138ae7...","[0.6539117097854614, 0.5391356348991394]"
1,3,[7f77b6d24f62a1ad1b469fdf4bc0a7f4],[0.3209395706653595]
2,12,[f5a390c1e5d342825b82bcc637a72e7a],[0.7968870401382446]
3,23,[d89b0bbdcbdb66475028b32eb8098dd4],[0.25853851437568665]
4,43,"[015f6154b8850c35a261314b4eb88846, 94101adfc2f...","[0.8686161041259766, 0.36814022064208984, 0.26..."


In [34]:
documents_categories_grouped_df.head()

,document_id_cat,category_id_list,confidence_level_cat_list
0,1,"[1706, 1603]","[0.47122037410736084, 0.03585372492671013]"
1,2,"[1705, 1708]","[0.6000000238418579, 0.4000000059604645]"
2,3,"[2003, 1403]","[0.5228370428085327, 0.039781082421541214]"
3,4,"[2003, 2006]","[0.9199999570846558, 0.07000000029802322]"
4,5,"[2006, 2003]","[0.9199999570846558, 0.07000000029802322]"


In [27]:
documents_meta_df=documents_meta_df.to_pandas()

In [35]:
documents_categories_grouped_df = pd.read_pickle('documents_categories_grouped_df.pkl')
documents_topics_grouped_df = pd.read_pickle('documents_topics_grouped_df.pkl')
# documents_entities_grouped_df.to_pickle('documents_entities_grouped_df.pkl')

In [30]:
documents_categories_grouped_df.dtypes

document_id_cat               int64
category_id_list             object
confidence_level_cat_list    object
dtype: object

In [32]:
documents_categories_grouped_df.head()

,document_id_cat,category_id_list,confidence_level_cat_list
0,1,"[1706, 1603]","[0.47122037410736084, 0.03585372492671013]"
1,2,"[1705, 1708]","[0.6000000238418579, 0.4000000059604645]"
2,3,"[2003, 1403]","[0.5228370428085327, 0.039781082421541214]"
3,4,"[2003, 2006]","[0.9199999570846558, 0.07000000029802322]"
4,5,"[2006, 2003]","[0.9199999570846558, 0.07000000029802322]"


In [36]:
documents_topics_grouped_df.head()

,document_id_top,topic_id_list,confidence_level_top_list
0,1,"[252, 35, 150, 276, 137]","[0.020726216956973076, 0.012773293070495129, 0..."
1,2,[216],[0.013028857298195362]
2,3,"[276, 107, 176, 160, 260, 283, 202, 156, 104, ...","[0.029913833364844322, 0.027589114382863045, 0..."
3,4,"[102, 75, 16, 244, 97, 239]","[0.08417084068059921, 0.08329997211694717, 0.0..."
4,5,"[75, 102, 244, 143, 130, 202, 16, 44, 64]","[0.10655508935451508, 0.08646664023399353, 0.0..."


In [37]:
documents_topics_grouped_df.dtypes

document_id_top               int64
topic_id_list                object
confidence_level_top_list    object
dtype: object

In [65]:
# ### Left join: each document has documents_meta, but some have no categories, topic or entity

documents_df1 = documents_meta_df.merge(documents_categories_grouped_df, how='left', left_on='document_id_doc', right_on='document_id_cat')

documents_df2 = documents_df1.merge(documents_topics_grouped_df, how='left', left_on='document_id_doc', right_on='document_id_top')

documents_df = documents_df2.merge(documents_entities_grouped_df, how='left', left_on='document_id_doc', right_on='document_id_ent')

In [21]:
evaluation = False
if evaluation:
    # TODO: no read need if whole script run 
    validation_set_df = pd.read_parquet(args.output_directory+"validation_set.parquet")

    users_to_profile = validation_set_df.loc[:, ['uuid_event']].drop_duplicates()

    validation_user_docs_to_ignore = validation_set_df.loc[:, ['uuid_event', 'document_id_promo']].drop_duplicates().rename(
        columns={'uuid_event': 'uuid_pv', 'document_id_promo': 'document_id_pv'})
else:
    events_df = cudf.read_csv(DATA_BUCKET_FOLDER+'events.csv', dtype=['int32','str','int32','int32','int32','str'], 
                                names=['display_id', 'uuid_event', 'document_id_event', 'timestamp_event', 'platform_event', 'geo_location_event'], header=0, na_values=['\\N', ''],
    keep_default_na=False,
    skiprows=1)
   
    events_df['event_country'] = events_df['geo_location_event'].str.slice(0,2)
    events_df['day_event'] = (events_df['timestamp_event'] / 1000 / 60 / 60 / 24).astype(int)
    # Drop rows with empty "geo_location"
    events_df = events_df.dropna(subset="geo_location_event")
   
    # Drop rows with empty "platform"
    events_df = events_df.dropna(subset="platform_event")
    
    promoted_content_df = cudf.read_csv(DATA_BUCKET_FOLDER+'promoted_content.csv', dtype=['int32', 'int32','int32','int32'], 
                                names=['ad_id', 'document_id_promo', 'campaign_id', 'advertiser_id'], header=0)

    clicks_test_df = cudf.read_csv(DATA_BUCKET_FOLDER+'clicks_test.csv', dtype=['int32', 'int32'], 
                                names=['display_id', 'ad_id'], header=0)
    test_set_df1 = clicks_test_df.merge(promoted_content_df, on='ad_id', how='left')
    test_set_df = test_set_df1.merge(events_df, on='display_id', how='left')
    users_to_profile = (np.unique(test_set_df['uuid_event'])).to_frame()
    test_users_docs_timestamp_to_ignore = test_set_df[['uuid_event', 'document_id_promo', 'timestamp_event']].drop_duplicates().reset_index(drop=True)

In [6]:
page_views_df = cudf.read_csv(DATA_BUCKET_FOLDER + "page_views.csv",
    names=['uuid_pv', 'document_id_pv', 'timestamp_pv', 'platform_pv', 'geo_location_pv', 'traffic_source_pv'],
    dtype={'uuid_pv': 'str', 'document_id_pv': 'int64', 'timestamp_pv': 'int64',
           'platform_pv': 'int64', 'geo_location_pv': 'str', 'traffic_source_pv': 'int64'},
    skiprows=1,
    keep_default_na=False)

In [24]:
# first apply (SELECT uuid_event FROM users_to_profile WHERE uuid_event = p.uuid_pv
page_views_train_df1 = page_views_df.loc[page_views_df['uuid_pv'].isin(users_to_profile['uuid_event'])]

In [25]:
page_views_train_df1.head()

,uuid_pv,document_id_pv,timestamp_pv,platform_pv,geo_location_pv,traffic_source_pv
273,7af903047336f7,5410,63553585,1,US>MI>505,2
290,5fe4adf60dce62,7012,42055742,1,CA>ON,1
440,eeb8c46c2eb234,7012,23428200,1,GB>H9,2
1241,1df91eb130564c,8226,24067562,3,GB>L2,2
1523,a0d65bd8d74d7e,10653,68132570,3,US>MN>613,2


In [50]:
# apply SELECT uuid_event FROM test_users_docs_timestamp_to_ignore  WHERE uuid_event = p.uuid_pv 
# AND document_id_promo = p.document_id_pv
page_views_joined_df1 = test_users_docs_timestamp_to_ignore.merge(page_views_df, how='outer', left_on=['uuid_event','document_id_promo'], right_on = ['uuid_pv','document_id_pv'])

In [51]:
page_views_joined_df1.head()

,uuid_event,document_id_promo,timestamp_event,uuid_pv,document_id_pv,timestamp_pv,platform_pv,geo_location_pv,traffic_source_pv
0,None,null,null,1fd5f051fba643,120,31905835,1,RS,2
1,None,null,null,8557aa9004be3b,120,32053104,1,VN>44,2
2,None,null,null,c351b277a358f0,120,54013023,1,KR>12,1
3,None,null,null,8205775c5387f9,120,44196592,1,IN>16,2
4,None,null,null,9cb0ccd8458371,120,65817371,1,US>CA>807,2


In [52]:
# apply AND p.timestamp_pv >= timestamp_event
page_views_joined_df2= page_views_joined_df1[page_views_joined_df1.timestamp_event >= page_views_joined_df1.timestamp_pv]

In [53]:
page_views_joined_df2.head()

,uuid_event,document_id_promo,timestamp_event,uuid_pv,document_id_pv,timestamp_pv,platform_pv,geo_location_pv,traffic_source_pv
10974475,e61e946589a203,1362397,60406889,e61e946589a203,1362397,60299153,1,US>PA>504,1


In [42]:
page_views_joined_df1.shape, test_users_docs_timestamp_to_ignore.shape, page_views_df.shape

((11047850, 9), (1048575, 3), (9999999, 6))

In [43]:
#test_users_docs_timestamp_to_ignore.loc[(test_users_docs_timestamp_to_ignore['uuid_event'].isin(page_views_df['uuid_pv'])) & (test_users_docs_timestamp_to_ignore['document_id_promo'].isin(page_views_df['document_id_pv']))]

In [92]:
print('Processing document frequencies...')

#both dfs here are cudf dataframes
documents_total = documents_meta_df.shape[0]

categories_docs_counts = documents_categories_df.groupby(['category_id']).count()
len(categories_docs_counts)

df_filenames_suffix = ''
if evaluation:
    df_filenames_suffix = '_eval'

with open(OUTPUT_BUCKET_FOLDER + 'categories_docs_counts' + df_filenames_suffix + '.pickle', 'wb') as output:
    pickle.dump(categories_docs_counts, output)

topics_docs_counts = documents_topics_df.groupby(['topic_id']).count()
len(topics_docs_counts)

with open(OUTPUT_BUCKET_FOLDER + 'topics_docs_counts' + df_filenames_suffix + '.pickle', 'wb') as output:
    pickle.dump(topics_docs_counts, output)
    

entities_docs_counts = documents_entities_df.groupby(['entity_id']).count()
len(entities_docs_counts)

with open(OUTPUT_BUCKET_FOLDER + 'entities_docs_counts' + df_filenames_suffix + '.pickle', 'wb') as output:
    pickle.dump(entities_docs_counts, output)

Processing document frequencies...


In [ ]:
print('Processing user profiles...')

null_to_empty_list = lambda x: x if isinstance(x, list) else []
null_to_empty_dict = lambda x: x if isinstance(x, dict) else {}
null_to_empty_string = lambda x: x if isinstance(x, str) else ''
null_to_minus_one = lambda x: -1 if np.isnan(x) else x
null_to_zero = lambda x: 0 if np.isnan(x) else x

In [93]:
def get_user_aspects(docs_aspects, aspect_docs_counts):
    docs_aspects_merged_lists = defaultdict(list)
  
    for doc_aspects in docs_aspects:
        #print(doc_aspects)
        for key in doc_aspects.keys():
            #print(key)
            docs_aspects_merged_lists[key].append(doc_aspects[key])
      
    docs_aspects_stats = {}
    
    for key in docs_aspects_merged_lists.keys():
        aspect_list = docs_aspects_merged_lists[key]
        tf = len(aspect_list)
        idf = math.log(documents_total / float(aspect_docs_counts[key]))
    
        confid_mean = sum(aspect_list) / float(len(aspect_list))
        print([tf*idf, confid_mean])
        docs_aspects_stats[key] = [tf*idf, confid_mean]
      
    return docs_aspects_stats


def generate_user_profile(docs_aspects_list, docs_aspects_confidence_list, aspect_docs_counts = categories_docs_counts):
    docs_aspects = []
    for doc_aspects_list, doc_aspects_confidence_list in zip(docs_aspects_list, docs_aspects_confidence_list):
        doc_aspects = dict(zip(doc_aspects_list, doc_aspects_confidence_list))
        docs_aspects.append(doc_aspects)
      
    user_aspects = get_user_aspects(docs_aspects, aspect_docs_counts)
    return user_aspects

In [ ]:
user_profiles_pdf['categories'] = page_views_by_user_pdf.apply(lambda x: generate_user_profile(x.category_id_lists, x.cat_confidence_level_lists), axis=1)
#user_profiles_pdf = pd.read_pickle('./outbrain/users_profile_pdf.pkl')

In [ ]:
list_len_udf = lambda x: len(x) if isinstance(x, list) else 0
user_profiles_pdf['views'] = page_views_by_user_pdf.loc[:, 'document_id_pv_list'].apply(list_len_udf)